In [5]:
from os import listdir
from os.path import isfile, join
#Para el corpus
import sys
sys.path.append('..')
from src.utils import *
from tqdm import tqdm 
from collections import defaultdict
import numpy as np
from operator import itemgetter
import json
import pickle
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [19]:
def get_subtittle():
    """
    Regresa una lista con el contenido de todos los archivos de un directorio

    Args:
        path (str): ruta de la carpeta
    """
    path = "../corpus/data"
    sub = defaultdict()
    info = defaultdict()
    onlyfiles = [f for f in listdir(path) if isfile(join(path, f))]

    for file in onlyfiles:
        with open(path+"/"+file, 'rb') as f:
            chanel = json.loads(f.read().decode('utf-8', 'replace'))
            for video in chanel:
                if 'subtitles' in video:
                    info[video["id"]] = []
                    for i, s in enumerate(video['subtitles']):
                        name = video["id"] + "-" + str(i)
                        sub[name] = get_tokens_clean(s['text'], stop_words=True)
                        info[video["id"]].append(s)
    return sub, info

In [20]:
sub, info = get_subtittle()

In [23]:
print(sub["68tp7jWrcjA-2"], info["68tp7jWrcjA"][2])

['frida', 'kahlo', 'andaba', 'regreso', 'escuela', 'ciudad', 'méxico'] {'start': '7.791', 'dur': '3.09', 'text': 'Frida Kahlo andaba de regreso de\nla escuela en la ciudad de México'}


In [37]:
# Convertimos los tokens de cada título en el formato requerido por gensim 
tagged_data = [TaggedDocument(d, [i]) for i, d in sub.items()]
tagged_data[:5]

[TaggedDocument(words=['adónde', 'vas', 'momo'], tags=['_DZF9EqExZM-0']),
 TaggedDocument(words=['débil'], tags=['_DZF9EqExZM-1']),
 TaggedDocument(words=['crees', 'podrás', 'sobrevivir'], tags=['_DZF9EqExZM-2']),
 TaggedDocument(words=['cumples', 'reglas', 'sociedad'], tags=['_DZF9EqExZM-3']),
 TaggedDocument(words=['vencido', 'encontramos', 'guarida'], tags=['_DZF9EqExZM-4'])]

Entrenamos

In [38]:
## Train doc2vec model
model = Doc2Vec(tagged_data, vector_size=10, window=2, min_count=1, workers=4, epochs = 100)
model.train(tagged_data, total_examples=model.corpus_count, epochs=model.epochs)
model.save("models/sub_doc2vec.model")

In [28]:
## Load saved doc2vec model
model= Doc2Vec.load("models/sub_doc2vec.model")
print(len(model.dv), len(tagged_data))

540324 540324


In [32]:
test_doc = get_tokens_clean("Receta de pastel con almendras")
model.dv.most_similar(positive=[model.infer_vector(test_doc)],topn=5)

[(381367, 0.954906165599823),
 (265583, 0.9480324387550354),
 (43245, 0.947913408279419),
 (95832, 0.9451383352279663),
 (252295, 0.9407374262809753)]

In [33]:
tagged_data[381367]

TaggedDocument(words=['nueva', 'zelanda'], tags=[381367])